In [1]:
import pandas as pd
corpus_classification = pd.read_csv('ru_subject_domain_classification.corpus', 
                          sep='\t', header=None)
corpus_table = corpus_classification.head(5).style.set_properties(**{'text-align': 'left'})    
corpus_table

,0,1
0,МАТЕМАТИКА,Алгоритм размещения элементов СБИС.
1,БИОЛОГИЯ,Восстановительное природопользование основа устойчивого развития.
2,БИОЛОГИЯ,Культура зародышей Paeonia anomala L.
3,ГЕОЛОГИЯ,Экологический мониторинг загазованных донных осадков.
4,БИОЛОГИЯ,Ультразвуковые методы определения газонасыщения биологических объектов.


In [2]:
corpus_classification.rename({0:'label', 1:'x'}, axis=1, inplace=True)
print(corpus_classification.label.value_counts())

БИОЛОГИЯ      1031
ГЕОЛОГИЯ       977
МАТЕМАТИКА     670
Name: label, dtype: int64


In [3]:
corpus_table = corpus_classification.head(5).style.set_properties(**{'text-align': 'left'}) 
corpus_table

,label,x
0,МАТЕМАТИКА,Алгоритм размещения элементов СБИС.
1,БИОЛОГИЯ,Восстановительное природопользование основа устойчивого развития.
2,БИОЛОГИЯ,Культура зародышей Paeonia anomala L.
3,ГЕОЛОГИЯ,Экологический мониторинг загазованных донных осадков.
4,БИОЛОГИЯ,Ультразвуковые методы определения газонасыщения биологических объектов.


In [4]:
'''
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from time import time
tic = time()
stemmer = SnowballStemmer("russian")
#print(preprocess_data(sent))
def preprocess_data(sent):
    #remove URLS
    sent = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', sent)
    sent = re.sub('@[^\s]+','', sent) #remove Users
    #smiles processing
    sent = re.sub('[:;^8=]{1}-?[)PD8o]+', 'positive_smile', sent)
    sent = re.sub('[:;^8=]{1}-?[(/|]+', 'negative_smile', sent)
    sent = sent.strip() # удаление пробелов по бокам
    sent = sent.lower() # lower-case
    sent = re.sub('[0-9]+', '', sent) # удаление цифр
     # удаление одноcимвольных токенов
    sent = re.sub(r'\b\w\b', '', sent)
    #удаление пунктуации
    sent = re.sub('[^A-Za-zА-Яа-я_\s]+', '', sent)
    # удаление стоп-слов
    sent = [x for x in sent.split() 
            if x not in stopwords.words('russian')]
    sent = [stemmer.stem(x) for x in sent] # cтемминг
    sent = ' '.join(sent) # cоединяем элементы списка
    toc = time()
    print(toc - tic)
    return sent
'''
import re
import pymorphy2
from nltk.corpus import stopwords
pattern = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()
def preprocess_data(msg):
    msg = re.sub(pattern, ' ', msg)
    tokens = []
    for token in msg.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    if len(tokens) > 2:
        tokens = ' '.join(tokens) # cоединяем элементы списка
        return tokens
    return None

In [5]:
sent = 'Восстановительное природопользование основа устойчивого развития.'
print(preprocess_data(sent))

восстановительный природопользование основа устойчивый развитие


In [6]:
corpus_classification['x'] = corpus_classification['x'].apply(preprocess_data)

In [7]:
corpus_table = corpus_classification.head(5).style.set_properties(**{'text-align': 'left'}) 
corpus_table

,label,x
0,МАТЕМАТИКА,алгоритм размещение элемент сбис
1,БИОЛОГИЯ,восстановительный природопользование основа устойчивый развитие
2,БИОЛОГИЯ,культура зародыш paeonia anomala l
3,ГЕОЛОГИЯ,экологический мониторинг загазованный донный осадок
4,БИОЛОГИЯ,ультразвуковой метод определение газонасыщение биологический объект


In [8]:
from sklearn.model_selection import train_test_split
x = corpus_classification.x
label = corpus_classification.label
x_train, x_test, label_train, label_test = train_test_split(x, label, test_size=.3)

In [9]:
#binary vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

binary_vectorizer = CountVectorizer(binary=True)
binary_vectorizer.fit(x)
binary_train = binary_vectorizer.transform(x_train)
binary_test = binary_vectorizer.transform(x_test)

binary_log_regr = LogisticRegression(solver = 'lbfgs', multi_class='auto')
binary_log_regr.fit(binary_train, label_train)
binary_predictions = binary_log_regr.predict(binary_test)
print("Точность модели после обучения на binary векторах:", 
      accuracy_score(label_test, binary_predictions))

Точность модели после обучения на binary векторах: 0.9166666666666666


In [10]:
#frequency vectorization
freq_vectorizer = CountVectorizer() 
freq_vectorizer.fit(x) 
freq_train = freq_vectorizer.transform(x_train)
freq_test = freq_vectorizer.transform(x_test)

freq_log_regr = LogisticRegression(solver = 'lbfgs', multi_class='auto')
freq_log_regr.fit(freq_train, label_train)
freq_predictions = freq_log_regr.predict(freq_test)
print("Точность модели после обучения на frequency векторах:", 
      accuracy_score(label_test, freq_predictions))

Точность модели после обучения на frequency векторах: 0.9340796019900498


In [11]:
#TF-IDF vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer() 
tfidf_vectorizer.fit(x) 
tfidf_train = tfidf_vectorizer.transform(x_train) 
tfidf_test = tfidf_vectorizer.transform(x_test)

tfidf_log_regr = LogisticRegression(solver = 'lbfgs', multi_class='auto')
tfidf_log_regr.fit(tfidf_train, label_train)
tfidf_predictions = tfidf_log_regr.predict(tfidf_test)
print("Точность модели после обучения на TF-IDF векторах:", 
      accuracy_score(label_test, tfidf_predictions))

Точность модели после обучения на TF-IDF векторах: 0.9241293532338308


In [12]:
#Проверка работы модели предсказаний
string = input("Введите краткое описание научной статьи: ")
print(f"Текст '{string}' относится к теме {tfidf_log_regr.predict(tfidf_vectorizer.transform([string]))[0]}.")

Введите краткое описание научной статьи: Геохимическая зональность скарново-золоторудных месторождений Западной Сибири.
Текст 'Геохимическая зональность скарново-золоторудных месторождений Западной Сибири.' относится к теме ГЕОЛОГИЯ.
